# OpenBeer - DataLoader - Notebook  
  
### 0. Load modules

In [ ]:
#Load modules and set seaborn styles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import glob
sns.set()

### I. Read Data

In [ ]:
#set columns from header row of csv file and path
columns_to_load = { "beers":["id","brewery_id","name","cat_id","style_id","abv","ibu","srm","upc","descript"],
                    "breweries":["id","name","address1","address2","city","state","code","country"], 
                    "categories":["id","cat_name"], 
                    "styles":["id","cat_id","style_name"],
                    "breweries_geocode":["id","brewery_id","latitude","longitude","accuracy"]}

dfs = {}
paths = [name for name in glob.glob("data\*.csv")]
files = [os.path.splitext(os.path.basename(name))[0] for name in glob.glob("data\*.csv")]
#read raw data for beers
for i,p in enumerate(paths):
    df_temp = pd.read_csv(p, index_col="id", usecols=columns_to_load[files[i]], na_values=[-1])
    dfs[files[i]] = df_temp.copy()

#explore top 5 rows if loaded correctly for each df
for f in files:
    print(f,dfs[f].info())

#assign each df to a separate variable for further exploration
df_beers_raw = dfs["beers"]
df_breweries_raw = dfs["breweries"]
df_breweries_geocode_raw = dfs["breweries_geocode"]
df_categories = dfs["categories"]
df_styles = dfs["styles"]

## Beers Cleaning and enhancement

In [ ]:
#making a copy of beer from raw
df_beers = df_beers_raw.copy()

### Overview:

In [ ]:
#Exploring Info, NaNs and Summary statistics

for c in df_beers.columns:
    print(df_beers[c].head())

print(df_beers.info(), "\n", df_beers.describe(), "\n", df_beers.isna().sum())

### Results:
Things to do:  
1. Change data types: ID will be replaced and still stored as objects, abv -> float, others->integers  
2. get rid of NaNs:all columns with NaNs exepted descript, cat, style will be dropped, cat + style NaNs will be extraced to be filled afterwards, descript will be stored separately  
3. exclude descript: second


In [ ]:


#drop beers without name or measurements
df_beers.dropna(subset=["name", "abv", "ibu", "srm", "upc"], axis = 0, inplace = True)

#extract NaN-rows in cat and style and save in csv
df_beers_cat_style_nan = df_beers[df_beers["cat_id"].isna() | df_beers[ "style_id"].isna()]
df_beers_cat_style_nan.to_csv("data\\temp\cat_style_nan.csv")

#extract and drop descript and save in csv
df_beers_descript = df_beers["descript"]
df_beers_descript.to_csv("data\\temp\\beers_descript.csv")
df_beers = df_beers.drop("descript", axis = 1)


#drop all others beers with NaNs
df_beers.dropna(axis=0, inplace=True)



### Explore again

In [ ]:

#get number of NaNs per column
print(df_beers.isna().sum())

#explore again
print(df_beers.info())
[print(df_beers[c].describe()) for c in df_beers.columns]

### Checking column per column

### 1. Name

In [ ]:
#explore name
print(df_beers["name"].head())
print(df_beers["name"].describe())

### Interesting Observations: There are beers with the same (probably generic) names. 
***Question: What are the top 10 most used names for beers? And how often are they used in the sample? Are they really generic?***

In [ ]:
#top 10 names
#get occurance of the 10th-often beer name
occurance_10th_value = df_beers["name"].value_counts().sort_values().tail(10).head(1)

#form df with top 10 names and duplicate occurances
df_grouped_names_often = df_beers.groupby("name").filter(lambda x: len(x) > occurance_10th_value-1)

#plot top 10 names and occurances
df_grouped_names_often["name"].value_counts().plot(kind="bar")
plt.show()

print("Answer:" ,"\n"," The top ten names are:", "\n")
print(df_beers["name"].value_counts().sort_values(ascending=False).head(10))
print("They are generic and used in maximum 48 times used in this sample (Pale Ale)")

#Edit: changing the name of the column due to duplicate names with brewery
df_beers["name_beer"] = df_beers["name"]
df_beers.drop("name", axis=1, inplace=True)

### 2. Brewery_id

In [ ]:
#explore brewery_id
print(df_beers["brewery_id"].head())
print(df_beers["brewery_id"].describe())

### Observations: Some breweries have a larger variety of beers in this sample. The values are IDs only. We should enhance the dataset with names of the breweries, longitude and latitude to look who, where which beers brew the beers are brewed

***Task: merge with brewery df on brewery_id, change id to name of the brewery, concat longitude and latitude, reorder meaningful***

In [ ]:
#make a copy from raw for breweries and geocodes
df_breweries = df_breweries_raw.copy()

#add long and lat from brewery geocodes on brewery_id to brewery table
df_breweries = pd.merge(df_breweries, df_breweries_geocode_raw, how="left", left_on = "id", right_on = "brewery_id")
df_breweries.drop("accuracy", axis=1, inplace = True)
df_breweries.reset_index(inplace=True)


#merge beer with brewery on brewery_id
df_beers_brewery = pd.merge(df_beers, df_breweries[["name", "city", "country", "latitude", "longitude"]] , how="left", left_on="brewery_id" , right_on=df_breweries.index)
df_beers_brewery["name_brewery"] = df_beers_brewery["name"]
df_beers_brewery.drop(["brewery_id","name"], axis=1, inplace=True)



#reorder columns
new_cols = ['name_beer', 'cat_id', 'style_id', 'abv', 'ibu', 'srm', 'upc', 'name_brewery', 'city',
       'country', 'latitude', 'longitude']
df_beers_brewery = df_beers_brewery[new_cols]
df_beers_brewery.head()

***Questions:  ***  
***1. How many different beers are brewed on average per brewery?***   
***2. Which are the top 5 most diverse breweries?***  
***3. What is the avg diversity of beers brewed per brewery per country?***  
***4. In which top 10 countries are the breweries most diverse?***

In [ ]:
#1 Group by brewery
df_grouped_brewery = df_beers_brewery.groupby("name_brewery" )
avg_beer_per_brewery = df_grouped_brewery["name_beer"].count().mean()

print("Answer 1: On average(mean) {:.2f} beers are brewed per brewery".format(avg_beer_per_brewery), "\n")

#2 used grouped_object
print("Answer 2: The top 5 most diverse breweries are:")
print(df_grouped_brewery["name_beer"].count().nlargest(5), "\n")

#3 calculate beers/country div by breweries/country
avg_div_per_country = df_beers_brewery.groupby("country")["name_beer"].nunique() / df_beers_brewery.groupby("country")["name_brewery"].nunique()
print("Answer 3: The average diversity in brewed beers per brewerie per country is:" ,"\n",avg_div_per_country.sort_values(ascending = False), "\n")
#4 get largest 10 and make comment on how many breweries are in a country
print("Answer 4: The top ten brewery-diversity countries are:","\n",avg_div_per_country.nlargest(10), "\n")
print("Comment on 4:")
for country in avg_div_per_country.nlargest(10).reset_index()["country"]:
    print(country + " has {} brewery(s)".format(df_beers_brewery.groupby("country").get_group(country)["name_brewery"].nunique()))